# [0] Imports

In [68]:
import Pkg;

using LinearAlgebra, Random, Gurobi, JuMP, Distributions, Plots, LazySets, Dates

In [69]:
LOW_JOBS = 35
HIGH_JOBS = 35;

In [70]:
times_pp_start = []
times_pp_end = []

times_mb_start = []
times_mb_end = []

times_op_start = []
times_op_end = []

ANSWERS = [] # record the actual routes
OBJECTIVES = [] # record the objective
;

In [71]:
for JOBS in LOW_JOBS:HIGH_JOBS
    
    # SETUP, doesn't count in time
    n_jobs = JOBS
    n_vehicles_jobs = n_jobs
    n_vehicles_coverage = 10
    T = 2 * n_jobs
    min_duration = 2
    max_duration = 6
    speed = 1000/6
    coverage_distance = 50
    size = 500
    step = 50;
    
    Random.seed!(1234)

    function create_cluster_sizes(jobs_total)
        Random.seed!(1234)
        jobs_created = 0
        cluster = []
        min_jobs_allowed = n_jobs >= 15 ? 3 : 2
            target_clusters = 5
            while jobs_created != jobs_total
                num_to_add = rand(min(min_jobs_allowed, jobs_total - jobs_created) : 
                    min(Int((n_jobs/target_clusters)÷1), jobs_total - jobs_created))
                jobs_created += num_to_add
                push!(cluster, num_to_add)
            end

        return cluster
    end;
    
    time_windows = []
    locations = rand(Uniform(0,size), 1, 2)
    work_load = []

    cluster = create_cluster_sizes(n_jobs);
    
    function create_time_windows_and_work_load(cluster_sizes, locations)
        Random.seed!(1234)
        locations = rand(Uniform(0,size), 1, 2)
        for size_c in cluster
            first = rand(Uniform(0,size), 1, 2)
            locations = vcat(locations, first)

            job_begins = rand(2:10)
            job_finish = rand((job_begins+min_duration):(job_begins+max_duration))
            push!(time_windows, [job_begins, job_finish])

            time_work = rand(min_duration:max(min_duration, job_finish - job_begins))
            push!(work_load, time_work)

            for neighbour in 1:(size_c-1)
                new_x = rand(Uniform(max(0,first[1]-20), min(first[1]+20, size)), 1, 1)
                new_y = rand(Uniform(max(0,first[2]-20), min(first[2]+20, size)), 1, 1)
                new = hcat(new_x, new_y)
                locations = vcat(locations, new)

                job_begins = rand(job_finish:min(T-min_duration-2, job_finish + 6))
                job_finish = rand((job_begins+min_duration):(min(job_begins+max_duration, T-2)))
                push!(time_windows, [job_begins, job_finish])

                time_work = rand(min_duration:min(max_duration, job_finish-job_begins))
                push!(work_load, time_work)
            end
        end

        return [time_windows, work_load, locations]
    end;
    
    time_windows, work_load, locations = create_time_windows_and_work_load(cluster, locations)
    distances = [LinearAlgebra.norm(locations[i, :] .- locations[j, :]) for i=1:n_jobs+1, j = 1:n_jobs+1];
    
    # NOW we start!
    push!(times_pp_start, datetime2unix(now()))
    function label_setting(n, travel_time, travel_distance, windows, load)
        N = [[1]] 
        T = [ [1] ] 
        C = [0.0]  

        L = [1]    

        current_state = 1
        total_state = 1

        while 1==1

            # STEP 1: Check whether we want to move on to checking the next path
            if (L[current_state] == n+2)
                current_state += 1
                if current_state > total_state
                    break
                else
                    continue
                end
            end

            # STEP 2: Enter the for loop.
            for i in 2:(n+2)

                # STEP 2-1: Check that the current node is not already inside our path.
                if ~(i in N[current_state])

                    #= STEP 2-2: Feasibility check. We must have enough time
                    to go from our current location (end time T[current_state]), plus
                    the travel time from that end node to the new node (travel_time[L[current_state], i]),
                    plus the amount of time it takes to work the OLD job (load[L[current_state]-1]),
                    plus the amount of time it takes to work the NEW job (load[i-1])

                    this must be smaller than or equal to the end time of the window for the NEW job (windows[i-1][2])
                    =#
                    if L[current_state] != 1
                        cur_time = last(T[current_state])
                        dist_nec = travel_time[L[current_state], i]
                        old_job_time_nec = load[L[current_state]-1] #out of our parameters this one has no depot pad
                        new_job_time_nec = load[i-1]
                        new_window_close = windows[i-1][2]

                        #if invalid, don't bother
                        if cur_time + dist_nec + old_job_time_nec + new_job_time_nec > new_window_close
                            continue
                        end
                    end

                    push!(N, copy(N[current_state]))
                    push!(N[total_state+1], i)

                    cur_time = last(T[current_state])
                    dist_nec = travel_time[L[current_state], i]
                    old_job_time = L[current_state] > 1 ? load[L[current_state]-1] : 0
                    new_window_start = windows[i-1][1]
                    new_times = copy(T[current_state])
                    push!(new_times, max(cur_time + dist_nec + old_job_time, new_window_start))
                    push!(T, new_times)

                    push!(C, C[current_state] + travel_distance[L[current_state], i])

                    push!(L, i)

                    total_state += 1

                end
            end

            current_state += 1 # concluded iteration for that state in mind. Now next one

            #= It's possible that we've just caught up to the final total state, meaning we didn't generate
            anything and can stop here. =#
            if current_state == total_state
                break
            end     
        end

        return N, C, T
    end
    
    distances_label = deepcopy(distances);
    distances_label = hcat(distances_label, distances_label[:, 1]);
    distances_label = vcat(distances_label, collect(push!(distances_label[:, 1], 0)'));
    travel_times = ceil.(distances_label / speed);
    windows_label = deepcopy(time_windows)
    push!(windows_label, [0, 100]);
    load_label = deepcopy(work_load)
    push!(load_label, 0);
    routes, cost, times = label_setting(n_jobs, travel_times, distances_label, windows_label, load_label);
    
    routes_keep = []
    C_routes = []
    T_routes = []

    # only if it ends in the depot is it useful
    for r in 1:length(routes)
        if (last(routes[r]) == n_jobs+2)
            push!(routes_keep, routes[r] .- 1)
            push!(C_routes, cost[r])
            push!(T_routes, times[r])
        end
    end
    
    n_routes = length(routes_keep);
    
    function route_time_combo(route, times)
        route_info = []
        for locindex in 1:length(route)-1
            #=
            Strategy:
            If you are not at a job location, you don't need to capture
            stationary information. Otherwise you do.

            Anyways we want data moving from one to the next.
            =#
            cur_loc = route[locindex]
            cur_time = times[locindex]
            time_to_start_moving = times[locindex]

            #stationary data comes first.
            if 1 < locindex < n_jobs + 2
                cur_job_num = cur_loc

                for time in cur_time:cur_time+work_load[cur_job_num]-1
                    route_detail = [ [cur_loc, time], [cur_loc, time+1], 0]
                    push!(route_info, route_detail)
                end

                time_to_start_moving += work_load[cur_job_num]
            end

            new_loc = route[locindex+1]
            #then if we move, we move.
            route_detail = [ [cur_loc, time_to_start_moving], [new_loc, times[locindex+1]], distances_label[cur_loc+1, new_loc+1]]
            push!(route_info, route_detail)
        end

        return route_info
    end;
    
    route_time_keep = [route_time_combo(routes_keep[i], T_routes[i]) for i in 1:n_routes];
    
    function compute_u(routes)
        Q = length(routes)
        u = [[0 for q in 1:Q] for i in 1:n_jobs]

        for rindex in 1:Q
            route = routes[rindex]
            for arc in route
                loc1 = arc[1][1]
                loc2 = arc[2][1]
                if (1 <= loc1 <= n_jobs)
                    u[loc1][rindex] = 1
                end
                if (1 <= loc2 <= n_jobs)
                    u[loc2][rindex] = 1
                end
            end
        end
        return u
    end
    
    u = compute_u(route_time_keep);
    # END parameters
    push!(times_pp_end, datetime2unix(now()))
    
    model_set = Model(Gurobi.Optimizer);
    set_optimizer_attribute(model_set, "OutputFlag", 0);
    
    # BEGIN modeling
    push!(times_mb_start, datetime2unix(now()))
    
    @variable(model_set, z[1:n_vehicles_jobs, 1:n_routes], Bin);
    @objective(model_set, 
        Min, 
        sum(sum(z[k,q]*C_routes[q] for q in 1:n_routes) for k in 1:n_vehicles_jobs))
    @constraint(model_set, 
        unique[i in 1:n_jobs],
        sum(sum(z[k,q]*u[i][q] for q in 1:n_routes) for k in 1:n_vehicles_jobs) == 1)
    @constraint(model_set, 
        driver[k in 1:n_vehicles_jobs], 
        sum(z[k,q] for q in 1:n_routes) == 1)
    
    # END modeling
    push!(times_mb_end, datetime2unix(now()))
    
    # BEGIN optimize
    push!(times_op_start, datetime2unix(now()))
    optimize!(model_set)
    push!(times_op_end, datetime2unix(now()))
    
    push!(OBJECTIVES, objective_value(model_set))
    
    z_val = value.(z);
    
    list_of_routes = []
    for vehicle_num in 1:n_vehicles_jobs
        for route_index in 1:n_routes
            if z_val[vehicle_num, route_index] > 0.99 && route_index > 1
                push!(list_of_routes, routes_keep[route_index])
            end
        end
    end
    push!(ANSWERS, list_of_routes)
end;

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


In [72]:
pp_arr = times_pp_end .- times_pp_start
mb_arr = times_mb_end .- times_mb_start
op_arr = times_op_end .- times_op_start
;

In [73]:
pp_arr

1-element Vector{Float64}:
 35.28500008583069

In [74]:
mb_arr

1-element Vector{Float64}:
 180.60700011253357

In [75]:
op_arr

1-element Vector{Float64}:
 366.33200001716614

In [76]:
total_arr = pp_arr .+ mb_arr .+ op_arr

1-element Vector{Float64}:
 582.2240002155304

In [77]:
for answer in ANSWERS
    for arcie in answer
        print(arcie[2:end-1])  
    end
    println()
end

[5, 6, 7, 8, 9][25, 26, 27][10, 11, 12, 13][1, 2, 3, 4][21, 22, 23, 24, 28][14, 15, 16, 17, 18, 19, 20][29, 30, 31, 32, 33, 34, 35]


In [78]:
OBJECTIVES

1-element Vector{Any}:
 3314.537527758451